# Prelude

## Import Packages

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%config InlineBackend.figure_format = 'svg'

In [3]:
import functions.init_logger
import logging
logger = logging.getLogger()
handler = logger.handlers[1]
handler.setLevel(logging.CRITICAL)

In [4]:
handler

In [5]:
import functions.add_path
import functions.et_condition_df as condition_df

import functions.et_helper as et_helper
import functions.plotnine_theme
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# this (for some reason) allows to edit text in illustrator 
# source: http://jonathansoma.com/lede/data-studio/matplotlib/exporting-from-matplotlib-to-open-in-adobe-illustrator/

matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.subplot.left'] =  0.2
from plotnine import *
from plotnine.data import *

from plotnine.options import figure_size as figsize


import be_load

/home/student/b/behinger/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# total width of a plot panel (I think it should be inch)
total_panel_width = 15

In [7]:
import os
print(os.getcwd())
if os.getcwd()[-4:] != 'code':
    try:
        os.chdir('./code')
    except:
        os.chdir('/net/store/nbp/users/behinger/projects/etcomp/code')


/net/store/nbp/users/behinger/projects/etcomp/code


In [8]:
import BLINK
import SMOOTH
import LARGE_GRID
import LARGE_and_SMALL_GRID
import FREEVIEW
import LUM
import MICROSACC
import CALIBRATION

reached exception:pyaudio
reached exception:calibration_methods
reached exception:glfw.GLFW_PRESS
reached exception:glfw.GLFW_KEY_SPACE
reached exception:glfw.GLFW_KEY_F
reached exception:glfw.glfwSetKeyCallback
reached exception:fingertip_calibration
reached exception:glfw.GLFW_PRESS
reached exception:natural_features_calibration
reached exception:calibration_methods
reached exception:glfw.GLFW_PRESS
reached exception:glfw.GLFW_KEY_SPACE
reached exception:glfw.GLFW_KEY_F
reached exception:glfw.glfwSetKeyCallback
reached exception:fingertip_calibration
reached exception:glfw.GLFW_PRESS
reached exception:natural_features_calibration


## Which algorithm?

In [9]:
#algorithm = 'hmmnosmooth_'
algorithm = '' # EM-algorithm
#algorithm = 'hmm_'

## Load Data

In [ ]:
(etsamples,etmsgs,etevents) = be_load.load_data(algorithm=algorithm)

File b'/net/store/nbp/projects/etcomp/results/preprocessed/el_cleaned_samples.csv' does not exist
2019-01-27 00:07:55 - be_load                                                           - CRITICAL - warning subject results et el not found
File b'/net/store/nbp/projects/etcomp/results/preprocessed/pl_cleaned_samples.csv' does not exist
2019-01-27 00:07:55 - be_load                                                           - CRITICAL - warning subject results et pl not found
File b'/net/store/nbp/projects/etcomp/backup/preprocessed/el_cleaned_samples.csv' does not exist
2019-01-27 00:07:55 - be_load                                                           - CRITICAL - warning subject backup et el not found
File b'/net/store/nbp/projects/etcomp/backup/preprocessed/pl_cleaned_samples.csv' does not exist
2019-01-27 00:07:55 - be_load                                                           - CRITICAL - warning subject backup et pl not found
File b'/net/store/nbp/projects/etcomp/conversion

In [ ]:
print(etsamples.subject.unique())
print("There are %i subjects"%(etsamples.subject.unique().shape))

In [ ]:
(ggplot(etsamples.query("(subject=='VP2'&smpl_time>1050&smpl_time<1050+30) | (subject=='VP4'&smpl_time>50 & smpl_time<(50+7))"),aes(x="smpl_time",y="gx",color="type"))
 +geom_point()
 +facet_grid("eyetracker~subject",scales="free_x")
+ theme(figure_size=((total_panel_width * (4/5)),(total_panel_width * (1/5)) * (4/3))))

In [ ]:
(ggplot(etsamples.query("(subject=='VP2'&smpl_time>1057&smpl_time<1057+7) | (subject=='VP4'&smpl_time>50 & smpl_time<(50+7))"),aes(x="smpl_time",y="gx",color="type"))
 +geom_point()
 +facet_grid("eyetracker~subject",scales="free_x")
+ theme(figure_size=((total_panel_width * (4/5)),(total_panel_width * (1/5)) * (4/3))))

## Interlude, check delay

In [ ]:
etmsgs_regress = etmsgs[etmsgs.exp_event.notnull()]

In [ ]:
tmp =etmsgs_regress.query('subject=="VP3"&condition!="Connect"').loc[:,['msg_time','eyetracker','exp_event','condition']]
tmp["id2"] = tmp.groupby("eyetracker").msg_time.rank(method='first')

In [ ]:
tmp = etsamples.query('subject=="VP2"')

In [ ]:
from functions.detect_saccades import interpolate_gaze

def wrapper_interpolate2(x,fs=500):
    #print("working on subject:%s"%(x.iloc[0]['subject']))
    
    tmp = interpolate_gaze(x,fs=fs)
    tmp.loc[tmp.is_blink!=0,['gx','gy']] = 0
    #tmp.loc[:,"eyetracker"] = x.iloc[0]['eyetracker']
    #tmp.loc[:,"subject"] = x.iloc[0]['subject']
    return(tmp)

In [ ]:
timeselect = '&smpl_time>1500&smpl_time<1550'

In [ ]:
etdat = wrapper_interpolate2(tmp.query('eyetracker=="el"'+timeselect)).dropna(axis=0).gx
pldat = wrapper_interpolate2(tmp.query('eyetracker=="pl"'+timeselect)).dropna(axis=0).gx

In [ ]:
xcorr = np.correlate(etdat,pldat,'full')

In [ ]:
plt.plot(xcorr)

In [ ]:
print('%i,%i'%(len(pldat)+1,np.argmax(xcorr)))

## Check durations of grid

In [ ]:
import MISC
grid_duration = MISC.grid_duration(etmsgs)

In [ ]:
print('Grids occured at: \n Small grid before: \t %.1fs [95%%range: %.1fs - %.1fs] \n Small grid after:\t %.1fs [95%%range: %.1fs - %.1fs]'%tuple(grid_duration.loc['el',['grid_time_before','grid_time_after']].values))

## Continue loading data

In [ ]:
if algorithm == 'hmm':
    # hmm has a very low threshold to detect smooth pursuit. We therefore use for the grid the nosmooth run of hmm
    (etsamples_grid,etmsgs_grid,etevents_grid) = be_load.load_data(algorithm='hmmnosmooth_')
else:
    # in all other cases, the '..._grid' is an alias
    etsamples_grid = etsamples
    etmsgs_grid = etmsgs
    etevents_grid = etevents

In [ ]:
namelist = ["large_grid","blink","smooth","microsaccade","lum","lum_binned","freeviewing","all_grids","tilt","shake","raw_fix_count_df"]

if 1==0:
    raw_large_grid_df = condition_df.get_condition_df(data=(etsamples_grid,etmsgs_grid,etevents_grid),condition='LARGE_GRID')

    blink= condition_df.get_condition_df(data=(etsamples,etmsgs,etevents),condition="BLINK")

    smooth = condition_df.get_condition_df(data=(etsamples,etmsgs,etevents),condition="SMOOTHPURSUIT")

    microsaccades = MICROSACC.detect_microsaccades(etsamples=etsamples,etmsgs=etmsgs,etevents=etevents)
    
    all_lum = LUM.process_lum(etsamples,etmsgs)
    all_lum_binned = LUM.bin_lum(all_lum)

    # Freeviewing
    raw_freeview_df, raw_fix_count_df = condition_df.get_condition_df(data=(etsamples_grid,etmsgs_grid,etevents_grid), condition='FREEVIEW') # loads the detection where all smooth pursuits are defined as fixations

    raw_all_grids_df = condition_df.get_condition_df(data=(etsamples_grid,etmsgs_grid,etevents_grid), condition='LARGE_and_SMALL_GRID')

    tilt = condition_df.get_condition_df(data=(etsamples,etmsgs,etevents),condition='TILT')
    shake = condition_df.get_condition_df(data=(etsamples,etmsgs,etevents),condition='SHAKE')
    datalist = [raw_large_grid_df,blink,smooth,microsaccades,all_lum,all_lum_binned,raw_freeview_df,raw_all_grids_df,tilt,shake,raw_fix_count_df]
    [dat.to_csv('/net/store/nbp/projects/etcomp/results/'+name+'.csv') for dat,name in zip(datalist,namelist)]
else:
    dataname = ["raw_large_grid_df","blink","smooth","microsaccades","all_lum","all_lum_binned","raw_freeview_df","raw_fix_count_df","raw_all_grids_df","tilt","shake"]
    for ix,name in  enumerate(namelist):
        print('loading '+name)
        exec(dataname[ix] +"= pd.read_csv('/net/store/nbp/projects/etcomp/results/'+name+'.csv')")

# Calibration Accuracy

## Load Data

In [ ]:
err = pd.DataFrame()

for subject in etsamples.subject.unique():
    err = pd.concat([err,CALIBRATION.pl_accuracy(subject),CALIBRATION.el_accuracy(subject)],ignore_index=True)
err.loc[:,'avg'] = err.avg.astype(float)
err.loc[:,'msg_time'] = err.msg_time.astype(float) 

In [ ]:
err.head()

In [ ]:
MISC.print_results(err,fields = ['avg'],round_to=2, agg_first_over_blocks=False)

In [ ]:
err.subject.unique()

## Results

In order to make nice plot panels, we have to set the size of the plots in a consistent manner.

In the following cell you can change the total standard with of the panels.

All plots will be scaled accordingly.

In [ ]:
from functions.et_helper import winmean_cl_boot
err.loc[:,'block'] = err.groupby(['subject','eyetracker']).cumcount()
err.loc[:,'group'] = err.subject + err.block.astype('str')
p_calib = (ggplot(err,aes(x="eyetracker",y="avg"))
               + geom_jitter(width=0.2,alpha=0.8, color='lightblue')
               #+geom_point(aes(group='group'),position=position_jitter(width=0.1),alpha=0.8,color='lightblue')
               #+geom_line(aes(group='group'),position=position_jitter(width=0.1),color='lightblue')
               + geom_hline(yintercept=[0.5,1.5], color='grey')
               + stat_summary(fun_data=winmean_cl_boot))


In [ ]:
p_calib + expand_limits(y=0)+ theme(figure_size=((total_panel_width * (1/5)),(total_panel_width * (1/5)) * (2/3)))

In [ ]:
 (ggplot(err,aes(x="subject",y="avg",color='eyetracker'))
               + geom_jitter(width=0.1)
               #+geom_point(aes(group='group'),position=position_jitter(width=0.1),alpha=0.8,color='lightblue')
               #+geom_line(aes(group='group'),position=position_jitter(width=0.1),color='lightblue')
               + geom_hline(yintercept=[0.5,1.5], color='grey')
               #+ stat_summary(fun_data=winmean_cl_boot)
 )+expand_limits(y=0)+  theme(figure_size=((total_panel_width * (3/10)),(total_panel_width * (1/5)) * (2/3)))


# Large Grid

## Methods

#### Single Trial view

In [ ]:
#%matplotlib ipympl
from functions import et_helper
et_helper.plot_around_event(etsamples_grid,etmsgs_grid,etevents_grid,raw_large_grid_df.query("eyetracker=='pl'&subject=='VP4'&block=='1'").iloc[5],plusminus=(-2,5))


In [ ]:
#%matplotlib ipympl
et_helper.plot_around_event(etsamples_grid,etmsgs_grid,etevents_grid,raw_large_grid_df.query("eyetracker=='el'&subject=='VP11'&block=='6'").iloc[9],plusminus=(-2,5))

In [ ]:
%matplotlib inline

## Results

#### Accuracy Table

In [ ]:
LARGE_GRID.make_table_accuracy_winmean(raw_large_grid_df)

In [ ]:
LARGE_GRID.make_table_accuracy(raw_large_grid_df)

## The sampling rates of the eye trackers

In [ ]:
bins = 1/np.linspace(600,50,50)
bincentres = [(bins[i]+bins[i+1])/2. for i in range(len(bins)-1)]

In [ ]:
tmp = etsamples.groupby(["eyetracker","subject"]).apply(lambda x: pd.DataFrame(data={'bin':bincentres,'n':np.histogram(np.diff(x.smpl_time),bins=bins)[0]}))

In [ ]:
ggplot(tmp.reset_index(),aes(x="1/bin",y="n",color="eyetracker"))+geom_point()

### Why the sampling rate of pupil labs is not really 240Hz (2x120Hz)
The sampling rates _seem_ fine, but this is due to their fusion algorithm. They simply enforce this sampling frequencies. In order to find "true" sampling frequencies, one needs to look at the pupil data which we do for one exemplary subject

In [ ]:
from functions.et_import import raw_pl_data


In [ ]:
def fs_get_ts(subject):
    tmp_dat = raw_pl_data(subject=subject)
    ts_0 = [p['timestamp'] for p in tmp_dat["pupil_positions"] if p["id"]==0]
    ts_1 = [p['timestamp'] for p in tmp_dat["pupil_positions"] if p["id"]==1]
    ts_both = [p['timestamp'] for p in tmp_dat["pupil_positions"]]
    return(ts_0,ts_1,ts_both)
def hist_samplingrate(ts,plot=False):
    bins = 1/np.linspace(1000,50,100)
    bins = np.linspace(0.001,0.01,100)
    bincentres = np.array([(bins[i]+bins[i+1])/2. for i in range(len(bins)-1)])
    
    h = np.histogram(np.diff(np.sort(ts)),bins=bins)
    
    if plot:
        plt.plot(1/bincentres,h[0],'o-')
    else:
            return(h,bincentres)

In [ ]:
ts_0,ts_1,ts_both = fs_get_ts("VP24")
hist_samplingrate(ts_0,plot=True)
hist_samplingrate(ts_1,plot=True)
hist_samplingrate(ts_both,plot=True)
plt.legend(['eye=0','eye=1','both eyes'])

In [ ]:
fs_all_sub = pd.DataFrame()
for subject in etmsgs.subject.unique():
    print("running subject {}".format(subject))
    ts_0,ts_1,ts_both = fs_get_ts(subject)
    h,bincentres = hist_samplingrate(ts_both)
    fs_all_sub = pd.concat([fs_all_sub,pd.DataFrame({'bins':bincentres,'n':h[0],'subject':subject})])

In [ ]:
ggplot(fs_all_sub,aes(x="bins",y="n",color="subject"))+geom_point()+geom_vline(xintercept=1/240)
#+xlim([110,700])

Why do we have two "sampling-rates" (sampling rate = 1/sample-distance)

(not that from the above graph we see that PL single eye cameras capture very consistently at 120Hz)

Image the A's and B's come from two cameras.(don't forget to recalibrate/sort your samples, pupil capture does not capture them sorted!!).
Pupillab fuses always two consecutive samples to the "C" samples. The timestamp is simply C1.ts = (A1.ts + B1.ts)
```
A1 _ _ _ _ B1 _ _ _ _ A2_ _ _ _ B2 _ _ _ _ A3 _ _ _ _ B3 ...

     C1         C2         C3         C4         C5      ...

A1 _ B1 _ _ _ _ _ _ _ A2 _ B2 _ _ _ _ _ _ _A3 _ B3 _ _ _  ...

  C1         C2          C3        C4        C5         ...
```

It is very clear that regardless of the actual sampling distance between the eyetrackers, we will always get an equally spaced $\Delta T_C$. This was in a previous plot visible as the 240Hz sampling rate.

But if we now look at the eye-camera wise $\Delta A/B$, the differences in time between two consecutive samples, in the first example we would get 240Hz, but in the second example we would get a very short intersampledistance (implying short sampling rates) and a very long one (implying very long sampling rates).Thats what we see in the upper plot, the sampling rate at round 140Hz is the large gap, the sampling rate at 550Hz is the short gap.

This explains why pupil labs **only seems to have 240Hz binocular but has not**


### Bad fusion of the eyes

In [ ]:
t0 = etmsgs.query("subject=='VP24'&eyetracker=='pl'&condition=='FREEVIEW'&pic_id==15").msg_time.values-4
td = 0.25

(
ggplot(etsamples.query("subject=='VP24'&eyetracker=='pl'&smpl_time<@t0+@td&smpl_time>@t0"),aes(x="smpl_time",y="gx"))
  + geom_point() 
  + ylab('gaze x-position [°]')
  + xlab("time [s]")
)

from this plot it is very clear, that the pupil lab eyes seem to be fused wrongly. If we take the previous insight into account, it seems as if pupil labs is somehow not correctly fusing the samples. That is, during the saccade arround 444.95 the second sample of each pair should be either more to the left (therefore taking into account that the eye samples are not captured at equal time distances (the A _ B _ _ _ A _ B _ _ _ "rhythm")), or they the second of each pair should be higher, thus indicating a successful fusion of the eyetrackers.

It is not clear to us why this occurs. pupil labs official response to this problem is "interpolation is not necessary using 200Hz eyetrackers". We don't really understand this response and we don't think it is appropriate to solve this problem

In [ ]:
(
ggplot(etsamples.query("subject=='VP24'&eyetracker=='pl'&smpl_time<@t0+@td&smpl_time>@t0"),aes(x="smpl_time",y="gy"))
  + geom_point() 
  + ylab('gaze y-position [°]')
  + xlab("time [s]")
)

the y-dimension gaze shows how during the saccade along the x-dimension at ~444.95 the y-position jumps up/down for every other sample. It is unclear to us why this happens but clearly it shows bad fusion between the eye cameras.

### Written Accuracy Results

In [ ]:
# evaluate only central points
MISC.print_results(raw_large_grid_df.query("posx<12&posx>-12&posy>-12&posy<12"),fields = ['duration','accuracy','rms','sd'],round_to=3)

In [ ]:
MISC.print_results(raw_large_grid_df,fields = ['duration','accuracy','rms','sd'],round_to=3)

### Duration and Duration diff

In [ ]:
p_grid_dur = (ggplot(raw_large_grid_df,aes(x="duration"))
              + geom_density(aes(group='subject'),color='lightblue',alpha=0.8)
              + geom_density(color='black',size=1)
              + facet_grid("~eyetracker")+xlim([0,5]))

In [ ]:
p_grid_dur + theme(figure_size=((total_panel_width * (2/5)), (total_panel_width * (2/5)) * (2/5)))

In [ ]:
def difference_function(x):
    if x.shape[0] == 1:
        return np.nan
    else:
        return np.diff(x)

In [ ]:

diff_large_grid_df = raw_large_grid_df.groupby(["element","block","subject"],as_index=False).duration.agg(difference_function)

In [ ]:
from functions import plotnine_theme
ggplot(diff_large_grid_df,aes(x="duration"))+ geom_freqpoly(color='black',size=1,binwidth=0.001)+xlim([-0.25,0.25])

This bimodal distribution of differences between eyelink and pupillabs eyetracker has it's source in catch-up saccades. They are usually quite small and pupil labs does generally not recognize them. Therefore, in most trials with catch-up saccades, eyelink has a shorter fixation duration compared to pupil labs.

In [ ]:
from functions.et_helper import winmean
raw_large_grid_df.groupby(["subject","eyetracker"],as_index=False).duration.agg(winmean).groupby("eyetracker").agg(winmean)

#### Accuracy Plots

In [ ]:
p_grid_acc = LARGE_GRID.plot_accuracy(raw_large_grid_df)
p_grid_acc + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
p_grid_rms = LARGE_GRID.plot_accuracy(raw_large_grid_df,depvar='rms')
p_grid_rms + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
p_grid_sd = LARGE_GRID.plot_accuracy(raw_large_grid_df,depvar='sd')
p_grid_sd + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

#### Single Subject Grid

In [ ]:
LARGE_GRID.display_fixations(raw_large_grid_df, option='offset',input_subject="VP1",input_block='1')

#### Grid element centered plot

In [ ]:
p_grid_fixcenter = LARGE_GRID.display_fixation_centered(raw_large_grid_df)+coord_fixed(xlim=[-2,2],ylim=[-2,2])
p_grid_fixcenter + theme(figure_size=((total_panel_width * (2/5)), (total_panel_width * (2/5)) * (1/3)))

#### Position vs Accuracy

In [ ]:
from functions.et_helper import winmean
p_grid_ac2d=(ggplot(raw_large_grid_df.groupby(['eyetracker','posx','posy'],as_index=False).agg(winmean),aes(x="posx",y="posy",size="accuracy"))
 +geom_point()
 +facet_wrap("~eyetracker",ncol=1)
+coord_fixed()
+scale_size_continuous(breaks=[0.25,0.5,0.75,1]))

In [ ]:
p_grid_ac2d  + theme(figure_size=((total_panel_width * (3/10)), (total_panel_width * (3/10))))

In [ ]:
raw_large_grid_df.loc[:,'posXY'] = raw_large_grid_df.posx.astype('str') +  raw_large_grid_df.posy.astype('str')

In [ ]:
p_grid_tdist=(ggplot(raw_large_grid_df,aes(x="mean_gx",y="mean_gy",group="posXY"))
 +stat_ellipse(type='t',level=0.95)
 +facet_wrap("~eyetracker",ncol=1)
+coord_fixed())
#+scale_size_continuous(range=[0.1,5],breaks=[0.25,0.5,0.75,1,1.25,1.5]))

In [ ]:
p_grid_tdist  + theme(figure_size=((total_panel_width * (3/10)), (total_panel_width * (3/10))))

In [ ]:
p_grid_rms2d=(ggplot(raw_large_grid_df.groupby(['eyetracker','posx','posy'],as_index=False).agg(winmean),aes(x="posx",y="posy",size="rms"))
 +geom_point()
 +facet_wrap("~eyetracker",ncol=1)
 +scale_size_area(breaks=[0.025,0.05,0.075,0.1,0.125])

 +coord_fixed())

In [ ]:
p_grid_rms2d + theme(figure_size=((total_panel_width * (3/10)), (total_panel_width * (3/10))))

In [ ]:
p_grid_sd2d=(ggplot(raw_large_grid_df.groupby(['eyetracker','posx','posy'],as_index=False).agg(winmean),aes(x="posx",y="posy",size="sd"))
 +geom_point()
 +facet_wrap("~eyetracker",ncol=1)
 +scale_size_area(breaks=[0.15,0.2,0.25,0.3,0.45])
 +coord_fixed())

In [ ]:
p_grid_sd2d + theme(figure_size=((total_panel_width * (3/10)), (total_panel_width * (3/10))))

#### Engbert vs. HMM
This can be thought of a precursor to compare different pupillabs detection algorithms e.g. (3D vs 2D)

if algorithm == '':
    etsamples_engbert = etsamples
    etmsgs_engbert = etmsgs
    etevents_engbert = etevents
    raw_large_grid_df_engbert = raw_large_grid_df
    
    (etsamples_hmm,etmsgs_hmm,etevents_hmm) = be_load.load_data(algorithm='hmmnosmooth_')
    raw_large_grid_df_hmm = condition_df.get_condition_df(data=(etsamples_hmm,etmsgs_hmm,etevents_hmm),condition='LARGE_GRID')
else:
    etsamples_hmm = etsamples
    etmsgs_hmm = etmsgs
    etevents_hmm = etevents
    raw_large_grid_df_hmm = raw_large_grid_df
    
    (etsamples_engbert,etmsgs_engbert,etevents_engbert) = be_load.load_data(algorithm='')
    raw_large_grid_df_engbert = condition_df.get_condition_df(data=(etsamples_engbert,etmsgs_engbert,etevents_engbert),condition='LARGE_GRID')

GRID_accuracy_table = pd.concat(
    [LARGE_GRID.make_table_accuracy_winmean(raw_large_grid_df_hmm).assign(    algorithm='hmm_nosmooth'),
     LARGE_GRID.make_table_accuracy_winmean(raw_large_grid_df_engbert).assign(algorithm='engbert'     )])
GRID_accuracy_table

## Statistical Model Tests (Beta)

In [ ]:
%load_ext rpy2.ipython
%Rpush raw_large_grid_df

In [ ]:
%%R
library(lme4)
raw_large_grid_df$et = factor(raw_large_grid_df$et)
contrasts(raw_large_grid_df$et)=c(-0.5,0.5)
mres <- lmer(accuracy ~ 1+et +  ( 1 + et | subject/block ) ,data = raw_large_grid_df)
summary(mres)

In [ ]:
%%R
#this cell takes very long to run!

#library(robustlmm)
#rmres <- rlmer(accuracy ~ 1+et +  ( 1 + et | subject/block ) ,data = raw_large_grid_df,verbose=0)
#summary(rmres)

## Supplementary

In [ ]:
# investigate on the position and properties of detected fixations
LARGE_GRID.display_fixations(raw_large_grid_df, option='fixations')

# Blinks

In [ ]:
blink.duration = blink.end_time - blink.start_time

blink.query("duration==0").shape # number of blinks removed because they have duration 0

In [ ]:
blink = blink.query("duration>0") # this removes the eyelin wrong blinks, they are in there since importing the data

## Results

#### Number of Blinks

In [ ]:
p_blink_count = BLINK.plot_count(blink)

In [ ]:
p_blink_count + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)  * (11/13))))

#### Duration of Blinks

In [ ]:
p_blink_dur = BLINK.plot_duration(blink)

In [ ]:
p_blink_dur + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)  * (11/13))))

# Smooth Pursuit

## Load Smooth Data

In [ ]:
# this loads the bayesian model results
cfg = {'redo':False} # should be specified in the beginning of the notebook
if cfg['redo']:
    smoothresult = SMOOTH.fit_bayesian_model(etsamples,etmsgs,etevents)
    SMOOTH.save_smooth(smoothresult)
else:
    smoothresult = SMOOTH.load_smooth()

## Methods

The model: Changepoint $\tau$ with  $pos  = \beta_{const}$ if $t<\tau$ and $pos = \beta_{const} + time * \beta_{velocity}$ if $t>=\tau$

#### Single Trial Modelfit

In [ ]:
smooth_stanmodel = SMOOTH.compileModel() # to facilitate plotting 

In [ ]:
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [(total_panel_width * (2/5)), (total_panel_width * (2/5)) * (3/4)]
plt.figure()

In [ ]:
fitA = SMOOTH.plot_single_trial(etsamples,etmsgs,None,'VP23','el',10,3,smooth_stanmodel)
fitB = SMOOTH.plot_single_trial(etsamples,etmsgs,None,'VP23','pl',10,3,smooth_stanmodel)

In [ ]:
fitB = SMOOTH.plot_single_trial(etsamples,etmsgs,None,'VP3','el',15,5,smooth_stanmodel)

Main Results follow. First latency, then estimated velocity

## Results

#### Initial Latency

In [ ]:
import SMOOTH

p_smooth_tau = SMOOTH.plot_modelresults(smoothresult,field="taumean",option="")
p_smooth_tau + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

#### Velocity

In [ ]:
p_smooth_vel = SMOOTH.plot_modelresults(smoothresult,field="velomean",option="")
p_smooth_vel + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
tmp = smoothresult.dropna(subset = ['velomean', 'taumean']).drop(["level_2",'0','summary'],axis=1)

In [ ]:
MISC.print_results(tmp, fields = ['taumean'], agg_first_over_blocks=True,round_to=3)

In [ ]:
MISC.print_results(tmp, fields = ['velomean'], agg_first_over_blocks=True,round_to=1)

#### Catchup Saccade Amplitudes

In [ ]:
p_smooth_catchup = SMOOTH.plot_catchup_amplitudes(smooth)+expand_limits(y = 0)
p_smooth_catchup + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
exec('smooth' +"= pd.read_csv('/net/store/nbp/projects/etcomp/results/smooth.csv')")

In [ ]:
smooth_saccade = smooth.query("type=='saccade'       & condition=='SMOOTH' & exp_event=='trialstart'") 
smooth_saccade = smooth_saccade.groupby(["subject","et","block",'trial',"angle","vel"],as_index=False).agg({'amplitude':sum,'duration':sum})
smooth_saccade_agg = smooth_saccade.groupby(["subject","et","block","angle","vel"],as_index=False).agg({'amplitude':et_helper.winmean,'duration':et_helper.winmean})
smooth_saccade_agg_agg = smooth_saccade_agg.groupby(["subject","et","angle","vel"],as_index=False).agg({'amplitude':et_helper.winmean,'duration':et_helper.winmean})
    

In [ ]:
ggplot(smooth_saccade_agg_agg,aes(x="vel",y="amplitude"))+stat_summary()+facet_wrap("et")

In [ ]:
ggplot(smooth_saccade_agg_agg,aes(x="angle",y="amplitude",color="vel",group="vel"))+stat_smooth(se=False)+facet_wrap("et")

In [ ]:
ggplot(smooth_saccade_agg_agg,aes(x="angle",y="duration",color="vel",group="vel"))+stat_smooth(se=False)+facet_wrap("et")

# Microsaccades

## Results

In [ ]:
ggplot(microsaccades,aes(x='amplitude'))+geom_freqpoly()

In [ ]:
# 6 blocks x 20s
p_microsacc_rate = MICROSACC.plot_default(microsaccades,subtype="count/(6*20)")+ylab("microsaccaderate [1/s]")
p_microsacc_rate + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
p_microsacc_amp = MICROSACC.plot_default(microsaccades,subtype="amplitude winmean")

In [ ]:
p_microsacc_amp +expand_limits(y=0)+ theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
MICROSACC.plot_densities(microsaccades)

In [ ]:
 MICROSACC.plot_mainsequence(microsaccades)+ylim((0,150))+xlim((-1.5,0.5))+ theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (8/4)))


# Pupil Dilation

## Results

In [ ]:
p_lum_time = LUM.plot_time_all(all_lum_binned)

In [ ]:
p_lum_time + theme(figure_size=((total_panel_width * (1/3)), (total_panel_width * (1/3)) * (5/6)))

In [ ]:
p_lum_all =LUM.plot_mean(all_lum)

In [ ]:
p_lum_all + theme(figure_size=((total_panel_width * (1/3)), (total_panel_width * (1/3)) * (5/6)))

In [ ]:
mean_lum = LUM.calc_mean(all_lum)
diff_lum = mean_lum.query("lum>0").groupby(["lum","block","subject"],as_index=False).pa_norm.agg(np.diff)
diff_lum.loc[:,'pa_norm'] = pd.to_numeric(diff_lum.loc[:,'pa_norm'])
sublist = diff_lum.query("lum>210").groupby("subject").agg("mean").sort_values(by="pa_norm").index

subject_cat = pd.api.types.CategoricalDtype(categories=sublist, ordered=True)

# Cast the existing categories into the new category. Due to a bug in pandas
# we need to do this via a string.
diff_lum.loc[:,'subject'] = diff_lum['subject'].astype(str).astype(subject_cat)

In [ ]:
diff_piv = diff_lum.pivot_table(index=["subject","block"],values="pa_norm",columns="lum")

In [ ]:
diff_piv.columns

In [ ]:
diff_piv = diff_piv.reset_index().melt(id_vars=["block","subject",64.0],value_vars=[128.0,192.0,255.0])

In [ ]:
diff_piv.columns = diff_piv.columns.astype(str)

In [ ]:
p_lum_individ2 = (ggplot(diff_piv.groupby(['subject','lum'],as_index=False).agg(winmean),aes(x="64.0",y="value",group="lum",color="subject",alpha='lum'))
 +geom_abline(intercept=0,slope=1,color='gray')
 +stat_summary()
 +scale_alpha_continuous(breaks=[128,192,255],range=[0.5,1])
 + coord_fixed(xlim=[-0.1,0.1],ylim=[-0.1,0.1]))

In [ ]:
p_lum_individ2 + theme(figure_size=((total_panel_width * (1/3)), (total_panel_width * (1/3)) * (5/6)))

In [ ]:
p_lum_individ = (ggplot(diff_lum,aes(x="lum",y="pa_norm",color="subject",group="subject"))
 +stat_summary(fun_y=winmean,geom="point")
 +stat_summary(fun_y=winmean,geom="line",linetype='dashed')
 +xlab('luminance [%max]')
 +stat_summary(aes(group=1),fun_data=winmean_cl_boot,size=1,color='grey',position=position_nudge(x=5))
 +stat_summary(aes(group=1),fun_y=winmean,geom='line',size=1,color='grey',position=position_nudge(x=5))
 +ylab("normed pupil size difference EL-PL"))

In [ ]:
p_lum_individ + theme(figure_size=((total_panel_width * (1/3)), (total_panel_width * (1/3)) * (5/6)))

In [ ]:
from functions.et_helper import winmean_cl_boot
ggplot(diff_lum,aes(x="factor(round(lum/255*100))",y="pa_norm",color="subject",group="subject"))+stat_summary(fun_data=winmean_cl_boot,position=position_dodge(width=0.4))+xlab('luminance [%max]')+ylab("pupil size EL-PL")  + theme(figure_size=((total_panel_width * (1/3)), (total_panel_width * (1/3)) * (5/6)))

In [ ]:
#etsamples_100hz = etsamples_100hz.query("smpl_time>%.2f"%(max(etsamples_100hz.groupby(["eyetracker"]).smpl_time.agg(min))))

In [ ]:
# currently not working
#etsamples_100hz.loc[etsamples_100hz.eyetracker=='el','pa_diff']=etsamples_100hz.query("eyetracker=='el'").pa.values-etsamples_100hz.query("eyetracker=='pl'").pa.values

In [ ]:
#currently not working
#ggplot(etsamples_100hz.loc[1:35000].query("eyetracker=='el'"),aes(x="pa",y="pa_diff"))+geom_point()

# Freeviewing

## Results

In [ ]:
# plot the fixations as a heatmap
# TODO annotation how many fixations from how many pictures are used for each eyetracker
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [(total_panel_width * (2/5)), (total_panel_width * (2/5)) * (3/4)]
p_free_heatmap = FREEVIEW.plot_heatmap(raw_freeview_df,raw_fix_count_df, only_horizontal_heatmap=True)

In [ ]:
# plot fixation counts
p_free_nfix = FREEVIEW.plot_number_of_fixations(raw_fix_count_df, option=None)

In [ ]:
p_free_nfix + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))

In [ ]:
import MISC

In [ ]:
raw_fix_count_df.columns


In [ ]:
MISC.print_results(raw_fix_count_df, fields = ['fix_counts'], agg_first_over_blocks=False,round_to=1)

In [ ]:
FREEVIEW.plot_scanpath(etsamples,etmsgs,subject='VP24', pic_id=4)

In [ ]:
plt.figure()
FREEVIEW.plot_scanpath(etsamples,etmsgs,subject='VP24', pic_id=15)

In [ ]:
# plot fixation durations
p_free_fixdur = FREEVIEW.plot_fixation_durations(raw_freeview_df)
p_free_fixdur #25ms bins

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_
raw_freeview_df.groupby(["eyetracker","subject"]).agg("count").groupby("eyetracker").agg(["sum","mean",percentile(25),percentile(75)])
#def print_results(df,fields = ['duration','accuracy','rms','sd'], round_to=2, agg_first_over_blocks=True):

In [ ]:
freeview_data = pd.DataFrame()

for sub in etmsgs.subject.unique():
    fromTList = etmsgs.query("pic_id>-1&eyetracker=='pl'& subject==@sub ").msg_time.values - 6
    toTList = fromTList + 6
    for fromT,toT in zip(fromTList,toTList):
        freeview_data = pd.concat([freeview_data,etevents.query("start_time>@fromT & start_time<=@toT   & subject==@sub ")])

In [ ]:
freeview_data[["type","eyetracker","subject","duration"]].groupby(["type","eyetracker","subject"]).agg("count").groupby(["type","eyetracker"]).agg(["sum","mean",percentile(25),percentile(75)])

In [ ]:
MISC.print_results(freeview_data.query("type=='saccade'"), fields = ['amplitude'], agg_first_over_blocks=False,round_to=2)

In [ ]:
ggplot(freeview_data.query("type=='saccade'"),aes(x="amplitude",color="eyetracker"))+geom_freqpoly(binwidth=0.25)

In [ ]:
MISC.print_results(raw_freeview_df, fields = ['duration'], agg_first_over_blocks=True,round_to=3)

In [ ]:
from functions.detect_saccades import interpolate_gaze
def wrapper_interpolate(x,fs=120):
    #print("working on subject:%s"%(x.iloc[0]['subject']))
    
    tmp = interpolate_gaze(x,fs=fs)
    tmp.loc[tmp.is_blink!=0,['gx','gy']] = np.nan
    #tmp.loc[:,"eyetracker"] = x.iloc[0]['eyetracker']
    #tmp.loc[:,"subject"] = x.iloc[0]['subject']
    return(tmp)

In [ ]:
subject = 'VP4'
elsamples_noclean,elmsgs_noclean,elevents_noclean = et_helper.load_file(subject=subject,cleaned=False,et='el')
plsamples_noclean,plmsgs_noclean,plevents_noclean = et_helper.load_file(subject=subject,cleaned=False,et='pl')
etsamples_noclean = pd.concat([elsamples_noclean.assign(eyetracker='el',subject=subject),plsamples_noclean.assign(eyetracker='pl',subject=subject)])
etmsgs_noclean    = pd.concat([elmsgs_noclean.assign(eyetracker='el',subject=subject),   plmsgs_noclean.assign(eyetracker='pl',subject=subject)])
etevents_noclean  = pd.concat([elevents_noclean.assign(eyetracker='el',subject=subject) ,plevents_noclean.assign(eyetracker='pl',subject=subject)])

In [ ]:
etsamples_noclean,etmsgs_noclean,etevents_noclean = be_load.regress_eyetracker(etsamples_noclean,etevents_noclean,etmsgs_noclean,subject=subject)

In [ ]:
freeviewtime = etmsgs_noclean.query("trial<4&condition=='FREEVIEW'&block=='3'")
freeviewtime

In [ ]:
etsamples_100hz_el  = wrapper_interpolate(etsamples_noclean.query("eyetracker=='el'&smpl_time>@freeviewtime.iloc[0].msg_time&smpl_time<=@freeviewtime.iloc[2].msg_time"),fs=1000)

etsamples_noclean.loc[etsamples_noclean.eyetracker=='pl','smpl_time'] = etsamples_noclean.query("eyetracker=='pl'").smpl_time + 0.01
etsamples_100hz_pl  = wrapper_interpolate(etsamples_noclean.query("eyetracker=='pl'&smpl_time>@freeviewtime.iloc[3].msg_time&smpl_time<=@freeviewtime.iloc[5].msg_time"),fs=1000)



In [ ]:
etsamples_100hz_diff = etsamples_100hz_el
etsamples_100hz_diff.loc[:,'gx'] = etsamples_100hz_el.gx - etsamples_100hz_pl.gx
etsamples_100hz_diff.loc[:,'gy'] = etsamples_100hz_el.gy - etsamples_100hz_pl.gy

In [ ]:
# ggplot(etsamples_100hz_diff,aes(x="smpl_time",y="gx"))+geom_point()
# This plot takes long time to plot and I therefore disabled it
# we still have delay here, the spikes are from saccades. we need to figure this one out

In [ ]:
if 1==0:
    # This cell is just there to check whether our alignment is correct. the pupillabs given 10ms look very good.
    timesel = 'smpl_time>13222&smpl_time<13222.5' # VP3
    timesel = 'smpl_time>7.32&smpl_time<17.35'
    timesel = 'smpl_time>17927&smpl_time<17929'
    ggplot(pd.concat([etsamples_100hz_el.assign(eyetracker='el'),etsamples_100hz_pl.assign(eyetracker='pl')]).query(timesel),aes(x="smpl_time",y="gx",color="eyetracker"))+geom_point()
    #ggplot(pd.concat([etsamples_el.assign(eyetracker='el'),etsamples_pl.assign(eyetracker='pl')]).query(timesel),aes(x="smpl_time",y="gx",color="eyetracker"))+geom_point()

# Small Grid

In [ ]:
p_grid_time = LARGE_and_SMALL_GRID.plot_accuracy(raw_all_grids_df, option='final_figure')+expand_limits(y=0)
p_grid_time + theme(figure_size=((total_panel_width * (2/5)), (total_panel_width * (2/5)) * (1/3)))

In [ ]:
#LARGE_and_SMALL_GRID.plot_accuracy(raw_all_grids_df, option='subjectvariance')

In [ ]:
all_grids_winmean = raw_all_grids_df.groupby(['condition','subject','et','block'], as_index=False).agg(winmean).groupby(['condition', 'subject','et'], as_index=False).agg(winmean)

%%R
library(lme4)
library(lmerTest)
library(dplyr)
all_grids_winmean$et = factor(all_grids_winmean$et)
all_grids_winmean$condition = factor(all_grids_winmean$condition)
#contrasts(all_grids_winmean$et)=c(-0.5,0.5)
#all_grids_winmean$et = all_grids_winmean$et == 'EyeLink'

mres <- lmer(accuracy ~ 1 + et*condition +  ( 1 | subject) ,data = all_grids_winmean)
print(summary(mres))
#print(anova(mres))
diff_all_grids_winmean = all_grids_winmean %>% group_by(subject,condition) %>% mutate(acc_diff = accuracy[et!='Pupil Labs'] -accuracy[et=='Pupil Labs'])
ggplot(diff_all_grids_winmean,aes(x=condition,y=acc_diff,color=interaction(subject),shape=et))+geom_point(position=position_dodge(width=0.5))+stat_summary()

# investigate on the position and properties of detected fixations
LARGE_and_SMALL_GRID.display_fixations(raw_all_grids_df, option='fixations')

# Shake

In [ ]:
from functions.et_helper import winmean,winmean_cl_boot
shakevalues = shake.shake_x.unique()
def find_min(list,target):
    return(list[np.argmin(np.abs(list-target))])
closest_grid = [find_min(raw_all_grids_df.query('condition!="GRID"').posx.unique(),x) for x in shakevalues]


data = pd.concat([shake,raw_all_grids_df.query("condition!='GRID'&posy==0")])
data = data.query("~(subject=='VP11' & block=='1')") # misunderstood instructions
data = data.query("~(subject=='VP1' & block=='3')") # one trial in block 3 was not rotated
data = data.query("~(subject=='VP19' & block=='1')") # one trial in block 3 was not rotated

In [ ]:
data = data.groupby(['eyetracker','condition','posx','subject'],as_index=False).agg(winmean)

In [ ]:
p_shake_gx=(ggplot(data,aes(x='posx',y='mean_gx-posx',color='condition'))+geom_point(alpha=0.1)+stat_summary(fun_data=winmean_cl_boot)+ facet_wrap("eyetracker"))
p_shake_gx + coord_cartesian(ylim=[-2.5,2.5])+ theme(figure_size=((total_panel_width  * (1/4)), (total_panel_width  * (1/2.) * (1/4))))

In [ ]:
p_shake_gy = (ggplot(data,aes(x='posx',y='mean_gy',color='condition'))+ geom_point(alpha=0.1)+stat_summary(fun_data=winmean_cl_boot)+ facet_wrap("eyetracker"))
p_shake_gy+ coord_cartesian(ylim=[-2.5,2.5])+ theme(figure_size=((total_panel_width  * (1/4)), (total_panel_width  * (1/2.) * (1/4))))


In [ ]:
# specify costumed minimal theme
import functions.plotnine_theme as mythemes

# save old theme and set the one for fixation plotting
#old_theme = theme_get()
#theme_set(mythemes.shake_rotate_theme)


shake_plot = (ggplot(data.query('condition=="SHAKE"'), aes(x='mean_gx-posx',y='mean_gy',shape='eyetracker',group="eyetracker",color='eyetracker',fill='posx'))
            + geom_point()
            + geom_path()
            + facet_wrap("subject",nrow=2)
            + coord_fixed(xlim=[-2.5,5],ylim=[-1.5,2.5])
            + mythemes.shake_rotate_theme
            + scale_fill_gradient(low='#000000',high='#CCCCCC')
            + theme(figure_size=((total_panel_width  * (1/2.)), (total_panel_width  * (1/2.) * (1/4)))))

shake_plot

# restore old theme
#theme_set(old_theme)

# Rotate

In [ ]:
from functions.et_helper import winmean

rotate = etmsgs.query('condition=="TILT"') # TILT is a missnomer and should be ROTATE
rotate.loc[:,'element'] = rotate.groupby(['subject','eyetracker','block','condition','exp_event']).cumcount()

rotate.loc[:,'angle_shift'] = rotate.angle.shift(1)
from functions.et_make_df import make_epochs
rotate_mean_all = pd.DataFrame()
for subject in etmsgs.subject.unique():
    if subject =='VP19':
        # As evident from the videos, VP 19 did not understand the task instructions. 
        continue
    for et in etmsgs.eyetracker.unique():
        
        subjectquery = 'subject==@subject & eyetracker==@et'
        
        try:
            rotate_epoch = make_epochs(etsamples.query(subjectquery),rotate.query(subjectquery), td=[-0.5,0])
            rotate_mean = rotate_epoch.groupby(['subject','eyetracker','block','angle_shift','element'],as_index=False).agg(winmean)
            rotate_mean.loc[:,'eyetracker'] = et
            rotate_mean.loc[:,'subject'] = subject
            rotate_mean_all = pd.concat([rotate_mean_all,rotate_mean])
        except:
            print('rotate: could not do sub:%s and et:%s'%(subject,et))

In [ ]:
rotate_mean = rotate_mean.query("~(subject=='VP22'&block=='1')")
rotate_mean = rotate_mean.query("~(subject=='VP19')")

In [ ]:

p_rotate_gx =(
 ggplot(rotate_mean_all.groupby(['angle_shift','subject','eyetracker'],as_index=False).agg(winmean),aes(x='angle_shift',y='gx',group="subject",color="subject"))
    +geom_path(linetype='dashed')
    +geom_point()
    +facet_wrap("eyetracker")
    +coord_cartesian(ylim=[-2,2])
    + theme(figure_size=((total_panel_width  * (1/4)), (total_panel_width  * (1/2.) * (1/4)))))



In [ ]:
p_rotate_gx

In [ ]:
p_rotate_gy = (
    ggplot(rotate_mean_all.groupby(['angle_shift','subject','eyetracker'],as_index=False).agg(winmean),aes(x='angle_shift',y='gy',group="subject",color="subject"))
    +geom_path(linetype='dashed')
    +geom_point()
    +facet_wrap("eyetracker")
    +coord_cartesian(ylim=[-2,2])
    + theme(figure_size=((total_panel_width  * (1/4)), (total_panel_width  * (1/2.) * (1/4)))))

In [ ]:
p_rotate_gy

In [ ]:
import rpy2.ipython
%reload_ext rpy2.ipython
%R require(ggplot2)
%R library("ggplot2")

df = rotate_mean_all.groupby(['subject','angle_shift','eyetracker'],as_index=False).agg(winmean)

In [ ]:
p = (ggplot(rotate_mean_all.groupby(['block','subject','angle_shift','eyetracker'],as_index=False).agg(winmean),
        aes(x='gx',y='gy',shape='eyetracker',group="eyetracker",color='eyetracker',fill='angle_shift'))
    +geom_point()
    +geom_path()
    +facet_grid("block~subject",scales='free')
    +coord_fixed()
    +scale_fill_gradient(low='#000000',high='#CCCCCC'))

#p

In [ ]:
# specify costumed minimal theme
import functions.plotnine_theme as mythemes




rotate_plot = (ggplot(df, aes(x='gx',y='gy',shape='eyetracker',group="eyetracker",color='eyetracker',fill='angle_shift'))
                + geom_point()
                + geom_path()
                + facet_wrap("subject",nrow=2)
                + coord_fixed(xlim=[-1.5,1.75],ylim=[-1.5,2.5])
                + scale_fill_gradient(low='#000000',high='#CCCCCC') 
                + mythemes.shake_rotate_theme
                + theme(figure_size=((total_panel_width * (1/2)), (total_panel_width * (1/2) * (1/4)))))

rotate_plot


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
def rotate_model(df):
    mod = smf.ols(formula='gx ~ angle_shift', data=df)
    res= mod.fit()
    return(res.params)
rotate_res = rotate_mean_all.groupby(["subject","eyetracker"]).apply(rotate_model)

In [ ]:
angle_shift = ggplot(rotate_res.reset_index(),aes(x="eyetracker",y="angle_shift")) + geom_line(aes(group='subject'), color='lightblue') + geom_point(color='lightblue')         

In [ ]:
angle_shift  + theme(figure_size=((total_panel_width * (1/5)), (total_panel_width * (1/5)) * (3/2)))